# Create Dataset
Aim to create the dataset of abstract for task classification.
From OpenAlex inverted_abstract will create a reconstructed string version of the abstract and classify this task in regard of the dataset the paper references

In [2]:
import pandas as pd
import requests
import numpy as np
import csv

datasets_context = {}
ds_reader = csv.DictReader(open('../../data/datasets.csv'))
for ds in ds_reader:
    datasets_context[ds["name"]] = ds["context"]
    
df = pd.read_csv("../../results/extracted_csv/paper_openalex.csv")

In [3]:
query_url = "https://api.openalex.org/works/doi"

paper_abstract = {}
with open("../../results/extracted_csv/paper_openalex.csv","r") as paper_csv:
    with open("../../data/abstract_dataset.csv","w") as abstract_file:
        abstract_file.write("title,doi,abstract,task")
        text = csv.reader(paper_csv,)
        header=next(text)
        for i,l in enumerate(text):
            ds = datasets_context[l[-1]]
            if not paper_abstract.get(l[1],None):
                r = requests.get(query_url+l[1])
                if r.status_code == 200:
                    r_json = r.json()
                    if r_json["abstract_inverted_index"]:
                        abstract = np.full(1000,"",dtype=object)
                        for w in r_json["abstract_inverted_index"]:
                            for indices in r_json["abstract_inverted_index"][w]:
                                abstract[indices] = ''.join(filter(str.isalnum, w)).lower()
                        abstract = abstract[abstract != ""]
                        str_abstract = ' '.join(abstract)
                        paper_abstract[l[1]] = (str_abstract,ds)
                        abstract_file.write(f'\n{l[0]},{l[1]},"{paper_abstract[l[1]][0]}",{ds}')
            elif ds != paper_abstract[l[1]][1]:
                abstract_file.write(f'\n{l[0]},{l[1]},"{paper_abstract[l[1]][0]}",{ds}')